### Convert ROIs from LASX roi file to ROIs generated via ImageJ macro 

**input:**
Path to LASX .roi file

**output:**
ImageJ macro file (.ijm) 

#### How to run:
- Specify path to LAS X generated ROIs file in: *file_path = r'roi_file.roi'*
- Run all cells.
- Output roi_file_fiji_rois.ijm will be created in same directory as input file.

In [19]:
# load packages

from xml.etree import ElementTree
import re
import os

In [58]:
# defined functions

# Function to read the file content
def read_file_as_text(file_path):
    try:
        with open(file_path, 'r') as file:
            return file.read()
    except Exception as e:
        return str(e)
    
# Function to extract X, Y coordinates from ROI
def extract_roi_coordinates(element):
    attachment = element.find('.//Attachment[@Name="ROIPixelSaveInfo"]')
    if attachment is not None:
        points = attachment.findall('.//P')
        coordinates = []
        for point in points:
            x = point.get('X')
            y = point.get('Y')
            coordinates.append((x, y))
        return coordinates
    return None

# Extracting coordinates for all ROIs
def extract_all_rois(root_element):
    makepolygon_strings = []
    for element in root_element.iter('Element'):
        if 'ROI.' in element.get('Name'):
            roi_coordinates = extract_roi_coordinates(element)
            if roi_coordinates:
                makepolygon_str = "makePolygon(" + ','.join(f"{x},{y}" for x, y in roi_coordinates) + ")"
                makepolygon_strings.append(makepolygon_str)
    return makepolygon_strings
    
def main(file_path):
    # Read file and parse XML content
    xml_content = read_file_as_text(file_path)
    tree = ElementTree.ElementTree(ElementTree.fromstring(xml_content))
    root = tree.getroot()

    # Get path info
    parent_path = os.path.dirname(file_path)
    filename = os.path.basename(file_path).rstrip('.roi')

    # Define the path for saving the file
    macro_file = filename + '_fiji_rois.ijm'
    save_path = os.path.join(parent_path, macro_file)

    # Extract ROIs
    imagej_rois = extract_all_rois(root)
    n_rois = len(imagej_rois)

    # Prepare macro text
    content = ''
    for i in imagej_rois:
        content += f"{i};\n"
        content += 'roiManager("Add");\n'

    # Write the content to the file
    with open(save_path, 'w') as f:
        f.write(content)

    print(f'File {macro_file} saved.')

In [59]:
# Set path to file
file_path = r'..\Projects_documentation\iu.roi'  # example

# Run code
if __name__ == "__main__":
    main(file_path)

File iu_fiji_rois.ijm saved.
